In [1]:
from SocialNetwork.social_network_environment import *
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
#from SocialNetwork.cascade import *
from Advertising.learners.Greedy_algorithm import *
from Advertising.learners.UCB_no_anything import *
from Advertising.learners.UCB_no_qualities import *

seed = 6789 
random.seed(seed)

exploration=4.2

deltas=[0.5,0.42,0.38,0.30,0.2,0.05]

social_network = social_network_environment()
social_network.init_from_json(json_path_network='Config/network.json', json_path_features='Config/features.json')

adversary_bids = []
for _ in range(0, 10): 
    adversary_i_bids = []
    for _ in range(0, 5):
        adversary_i_bids.append(random.randint(0, 4))
    adversary_bids.append(adversary_i_bids)

arms = arms_creation(seed = 30291, number_of_arms= 20)
n_rounds=10000
estimation_rounds=2000
all_bids = adversary_bids
vcg=VCG()
with open('Config/estimated_influence.json', 'r') as file:
    data = json.load(file)
    nodes_estimation = data.get('estimated_influence')
#nodes_estimation=compute_array_estimated_influence(social_network=social_network, rounds=estimation_rounds)

In [2]:
#Clairvoyant for point 4, only first position
all_exp_rewards=[]
for arm in arms:
    print("Doing arm "+ str(arm))
    exp_reward=0
    all_bids.insert(0, arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)
    for j in range(0, len(best_allocation)):
            temp_allocation = []
            temp_allocation = best_allocation[j]
            for k in range(0, len(best_allocation[j])):
                if temp_allocation[k].ad_id == 1 and k != 0:
                    temp_allocation[k].ad_id = None
                    best_allocation[j] = temp_allocation
    for i in range(1, n_rounds+1):  
        payments = vcg.payments(ad_allocation_list, best_allocation, social_network)
        active_nodes, reward_click = active_nodes_click(social_network, best_allocation, deltas, learner_id=1)
        reward_influence = 0
        seeds=np.where(active_nodes==1)[0]
        if len(seeds) != 0:
            for k in range(0, len(seeds)):
                reward_influence += nodes_estimation[seeds[k]]
            reward_influence = reward_influence/len(seeds)# + reward_influence)/2
        payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)
        reward = (reward_click+reward_influence-payments_tot) 
        reset_nodes(social_network)
    
        exp_reward = (exp_reward*(i-1) + reward)/i
    all_bids.pop(0)
    all_exp_rewards.append(exp_reward)

clairvoyant_q_f=max(all_exp_rewards)
index_q_f=np.argmax(all_exp_rewards)

Doing arm [4, 1, 1, 3, 0]
Doing arm [3, 3, 4, 2, 3]
Doing arm [1, 4, 3, 2, 0]
Doing arm [1, 3, 3, 1, 4]
Doing arm [1, 1, 2, 0, 4]
Doing arm [2, 4, 4, 3, 0]
Doing arm [4, 0, 0, 0, 1]
Doing arm [2, 1, 3, 0, 4]
Doing arm [4, 2, 2, 2, 4]
Doing arm [0, 4, 3, 3, 1]
Doing arm [2, 3, 4, 2, 4]
Doing arm [2, 2, 3, 3, 4]
Doing arm [0, 4, 4, 2, 1]
Doing arm [4, 2, 3, 4, 0]
Doing arm [2, 3, 2, 1, 1]
Doing arm [1, 4, 4, 2, 3]
Doing arm [0, 0, 4, 4, 4]
Doing arm [0, 0, 1, 4, 0]
Doing arm [3, 1, 2, 2, 3]
Doing arm [3, 4, 3, 3, 0]


In [3]:
#Clairvoyant for point 4, with all positions
all_exp_rewards=[]
for arm in arms:
    print("Doing arm "+ str(arm))
    exp_reward=0
    all_bids.insert(0, arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)
    for i in range(1, n_rounds+1):  
        payments = vcg.payments(ad_allocation_list, best_allocation, social_network)
        active_nodes, reward_click = active_nodes_click(social_network, best_allocation, deltas, learner_id=1)
        reward_influence = 0
        seeds=np.where(active_nodes==1)[0]
        if len(seeds) != 0:
            for k in range(0, len(seeds)):
                reward_influence += nodes_estimation[seeds[k]]
            reward_influence = reward_influence/len(seeds)# + reward_influence)/2
        payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)
        reward = (reward_click+reward_influence-payments_tot) 
        reset_nodes(social_network)
    
        exp_reward = (exp_reward*(i-1) + reward)/i
    all_bids.pop(0)
    all_exp_rewards.append(exp_reward)

clairvoyant_q_a=max(all_exp_rewards)
index_q_a=np.argmax(all_exp_rewards)

Doing arm [4, 1, 1, 3, 0]
Doing arm [3, 3, 4, 2, 3]
Doing arm [1, 4, 3, 2, 0]
Doing arm [1, 3, 3, 1, 4]
Doing arm [1, 1, 2, 0, 4]
Doing arm [2, 4, 4, 3, 0]
Doing arm [4, 0, 0, 0, 1]
Doing arm [2, 1, 3, 0, 4]
Doing arm [4, 2, 2, 2, 4]
Doing arm [0, 4, 3, 3, 1]
Doing arm [2, 3, 4, 2, 4]
Doing arm [2, 2, 3, 3, 4]
Doing arm [0, 4, 4, 2, 1]
Doing arm [4, 2, 3, 4, 0]
Doing arm [2, 3, 2, 1, 1]
Doing arm [1, 4, 4, 2, 3]
Doing arm [0, 0, 4, 4, 4]
Doing arm [0, 0, 1, 4, 0]
Doing arm [3, 1, 2, 2, 3]
Doing arm [3, 4, 3, 3, 0]


In [4]:
#Clairvoyant for point 5, only first position
all_exp_rewards=[]
for arm in arms:
    print("Doing arm "+ str(arm))
    exp_reward=0
    all_bids.insert(0, arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)
    for j in range(0, len(best_allocation)):
            temp_allocation = []
            temp_allocation = best_allocation[j]
            for k in range(0, len(best_allocation[j])):
                if temp_allocation[k].ad_id == 1 and k != 0:
                    temp_allocation[k].ad_id = None
                    best_allocation[j] = temp_allocation
    for i in range(1, n_rounds+1):  
        payments = vcg.payments(ad_allocation_list, best_allocation, social_network)
        active_nodes, total_reward = activate_cascade(social_network=social_network, ad_allocation_list=best_allocation, slot_prominence=deltas,learner_id=1)
        payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)
        reward = (total_reward-payments_tot) 
        reset_nodes(social_network)
        exp_reward = (exp_reward*(i-1) + reward)/i
    all_bids.pop(0)
    all_exp_rewards.append(exp_reward)

clairvoyant_nq_f=max(all_exp_rewards)
index_nq_f=np.argmax(all_exp_rewards)

Doing arm [4, 1, 1, 3, 0]
Doing arm [3, 3, 4, 2, 3]
Doing arm [1, 4, 3, 2, 0]
Doing arm [1, 3, 3, 1, 4]
Doing arm [1, 1, 2, 0, 4]
Doing arm [2, 4, 4, 3, 0]
Doing arm [4, 0, 0, 0, 1]
Doing arm [2, 1, 3, 0, 4]
Doing arm [4, 2, 2, 2, 4]
Doing arm [0, 4, 3, 3, 1]
Doing arm [2, 3, 4, 2, 4]
Doing arm [2, 2, 3, 3, 4]
Doing arm [0, 4, 4, 2, 1]
Doing arm [4, 2, 3, 4, 0]
Doing arm [2, 3, 2, 1, 1]
Doing arm [1, 4, 4, 2, 3]
Doing arm [0, 0, 4, 4, 4]
Doing arm [0, 0, 1, 4, 0]
Doing arm [3, 1, 2, 2, 3]
Doing arm [3, 4, 3, 3, 0]


In [5]:
#Clairvoyant for point 5, with all positions
all_exp_rewards=[]
for arm in arms:
    print("Doing arm "+ str(arm))
    exp_reward=0
    all_bids.insert(0, arm)
    ad_allocation_list = setup(bids=all_bids, n_bids=5)
    best_allocation = vcg.all_best_allocations(ad_allocation_list, social_network)
    for i in range(1, n_rounds+1):  
        payments = vcg.payments(ad_allocation_list, best_allocation, social_network)
        active_nodes, total_reward = activate_cascade(social_network=social_network, ad_allocation_list=best_allocation, slot_prominence=deltas,learner_id=1)
        payments_tot = calculate_total_payment(payments, social_network.categories, active_nodes)
        reward = (total_reward-payments_tot) 
        reset_nodes(social_network)
        exp_reward = (exp_reward*(i-1) + reward)/i
    all_bids.pop(0)
    all_exp_rewards.append(exp_reward)

clairvoyant_nq_a=max(all_exp_rewards)
index_nq_a=np.argmax(all_exp_rewards)
        

Doing arm [4, 1, 1, 3, 0]
Doing arm [3, 3, 4, 2, 3]
Doing arm [1, 4, 3, 2, 0]
Doing arm [1, 3, 3, 1, 4]
Doing arm [1, 1, 2, 0, 4]
Doing arm [2, 4, 4, 3, 0]
Doing arm [4, 0, 0, 0, 1]
Doing arm [2, 1, 3, 0, 4]
Doing arm [4, 2, 2, 2, 4]
Doing arm [0, 4, 3, 3, 1]
Doing arm [2, 3, 4, 2, 4]
Doing arm [2, 2, 3, 3, 4]
Doing arm [0, 4, 4, 2, 1]
Doing arm [4, 2, 3, 4, 0]
Doing arm [2, 3, 2, 1, 1]
Doing arm [1, 4, 4, 2, 3]
Doing arm [0, 0, 4, 4, 4]
Doing arm [0, 0, 1, 4, 0]
Doing arm [3, 1, 2, 2, 3]
Doing arm [3, 4, 3, 3, 0]


In [6]:
all_clairvoyants = []
all_clairvoyants.append(clairvoyant_q_f)
all_clairvoyants.append(clairvoyant_q_a)
all_clairvoyants.append(clairvoyant_nq_f)
all_clairvoyants.append(clairvoyant_nq_a)
all_indexes = []
all_indexes.append(index_q_f)
all_indexes.append(index_q_a)
all_indexes.append(index_nq_f)
all_indexes.append(index_nq_a)

In [7]:
print(all_clairvoyants)
print(all_indexes)

[0.24936147844443757, 0.48842019490079885, 1.7742390000000792, 2.1363950000000385]
[4, 14, 1, 14]


In [8]:
import json
with open('Config/clairvoyants.json', 'w') as file:
    to_write = {'clairvoyant_q_f': clairvoyant_q_f, 'clairvoyant_q_a': clairvoyant_q_a, 'clairvoyant_nq_f': clairvoyant_nq_f, 'clairvoyant_nq_a': clairvoyant_nq_a}
    json.dump(to_write, file)
    
